In [13]:
##### import sys
import re
import gc
import sys
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_log_error

sys.path.append("../")
from general.preprocess import data_preparation
from general.clf_wrappers import LgbWrapper
from features.f0 import features_set_f0

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
full_data, ntrain, ntest = data_preparation()

air_store dataframe shape: (829, 57)


In [6]:
trn = full_data[:ntrain]
tst = full_data[ntrain:]
print('train data size:', trn.shape, 'test data size:', tst.shape)

train data size: (252108, 123) test data size: (32019, 123)


In [7]:
TARGET = 'visitors'
FEATURES = features_set_f0()
SEED = 177
lgb_params = {}
lgb_params['n_estimators'] = 50
lgb_params['learning_rate'] = 0.1 # shrinkage_rate
lgb_params['metric'] = 'l1'          # or 'mae'
lgb_params['sub_feature'] = 0.34
lgb_params['bagging_fraction'] = 0.85 # sub_row
lgb_params['bagging_freq'] = 40
lgb_params['num_leaves'] = 512        # num_leaf
lgb_params['min_data'] = 500         # min_data_in_leaf
lgb_params['min_hessian'] = 0.05     # min_sum_hessian_in_leaf
lgb_params['verbose'] = 0
lgb_params['feature_fraction_seed'] = 2
lgb_params['bagging_seed'] = 3

In [9]:
from sklearn.metrics import mean_squared_log_error


def score_valid(y_true, y_valid):
    score = np.sqrt(mean_squared_log_error(y_true, y_valid))
    return score

def get_store_ids():
    df = pd.read_csv('../data/air_store_info.csv')
    return df['air_store_id']

In [40]:
def cross_validate(full_data, clf, seed, ntrain, ntest, features, target, nfolds=5):
    # define
    lst_vld_date = pd.to_datetime('2017-4-22')  # one day before test
    lst_v06_date = lst_vld_date - pd.DateOffset(days=38)
    lst_v33_date = lst_vld_date - pd.DateOffset(days=32)   
    step_by_days = 10
   # folds
    
    # split train/valid
    trn = full_data[:ntrain]
    tst = full_data[ntrain:]
    
    # setup train on test
    x_trn = trn[features].values
    y_trn = trn[target].values
    x_tst = tst[features].values
    
    # remove unnecessary data sets
    del full_data
    gc.collect()
    
    v06_scoring = np.zeros((nfolds,))
    v33_scoring = np.zeros((nfolds,))
    oof_tst = np.zeros((ntest,))
    oof_tst_fld = np.empty((ntest, nfolds))
    oof_score = [None] * 2

    for i in range(nfolds):
        # first date of each set
        v06_date = lst_v06_date - pd.DateOffset(days=(nfolds-i-1)*step_by_days) 
        v33_date = lst_v33_date - pd.DateOffset(days=(nfolds-i-1)*step_by_days)

        # setup train on valid
        x_tvd = trn[trn.visit_date < v06_date][features].values
        y_tvd = trn[trn.visit_date < v06_date][target].values
        assert(x_tvd.shape[0] == y_tvd.shape[0])
        
        x_v06 = trn[trn.visit_date.between(v06_date, v33_date - pd.DateOffset(days=1))][features].values
        y_v06 = trn[trn.visit_date.between(v06_date, v33_date - pd.DateOffset(days=1))][target].values
        
        x_v33 = trn[trn.visit_date.between(v33_date, v33_date + pd.DateOffset(days=32))][features].values
        y_v33 = trn[trn.visit_date.between(v33_date, v33_date + pd.DateOffset(days=32))][target].values
        # train on train_valid set and predict on v06/v33 set
        clf.train(x_tvd, y_tvd)
        v06_scoring[i] = score_valid(y_v06, clf.predict(x_v06))
        v33_scoring[i] = score_valid(y_v33, clf.predict(x_v33))

        # train on train_full set and predict on test
        clf.train(x_trn, y_trn)
        oof_tst_fld[:, i] = clf.predict(x_tst)

        del x_tvd, y_tvd, x_v06, x_v33
        gc.collect()
    
    oof_tst[:] = oof_tst_fld.mean(axis=1)
    oof_score[0] = np.mean(v06_scoring)
    oof_score[1] = np.mean(v33_scoring)

    sub = tst[['id', target]]
    sub[target] = oof_tst.reshape(-1, 1)
    
    v06 = None
    v33 = None

    return sub, v06, v33, oof_score

In [41]:
lgb_clf= LgbWrapper(seed=SEED, params=lgb_params)
results = cross_validate(
    full_data=full_data,
    clf=lgb_clf,
    seed=SEED,
    ntrain=ntrain,
    ntest=ntest,
    features=FEATURES,
    target=TARGET,
    nfolds=5,
)
print(1)

1


G:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [43]:
results[3]

[0.46617470884691, 0.48753824640516247]